In [1]:
import os
import gradio as gr
import requests
import datetime
from dotenv import load_dotenv
import json
import anthropic
from openai import OpenAI
import google.generativeai as genai
from nsepython import *
import math
from statistics import stdev

In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# Initialize API clients
anthropic_client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [4]:
# Model configurations
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
GEMINI_MODEL = "gemini-2.0-flash-exp"

In [5]:
# Stock data API endpoints
ALPHA_VANTAGE_API = "https://www.alphavantage.co/query"
ALPHA_VANTAGE_KEY = os.getenv("ALPHA_VANTAGE_KEY")

In [6]:
# ==================== TECHNICAL INDICATOR CALCULATIONS ====================

def calculate_sma(prices, period):
    """Calculate Simple Moving Average"""
    if len(prices) < period:
        return None
    return sum(prices[-period:]) / period


def calculate_ema(prices, period):
    """Calculate Exponential Moving Average"""
    if len(prices) < period:
        return None
    
    multiplier = 2 / (period + 1)
    ema = sum(prices[:period]) / period  # Start with SMA
    
    for price in prices[period:]:
        ema = (price * multiplier) + (ema * (1 - multiplier))
    
    return ema


def calculate_rsi(prices, period=14):
    """Calculate Relative Strength Index"""
    if len(prices) < period + 1:
        return None
    
    gains = []
    losses = []
    
    for i in range(1, len(prices)):
        change = prices[i] - prices[i-1]
        if change > 0:
            gains.append(change)
            losses.append(0)
        else:
            gains.append(0)
            losses.append(abs(change))
    
    if len(gains) < period:
        return None
    
    avg_gain = sum(gains[-period:]) / period
    avg_loss = sum(losses[-period:]) / period
    
    if avg_loss == 0:
        return 100
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi


def calculate_macd(prices):
    """Calculate MACD (Moving Average Convergence Divergence)"""
    if len(prices) < 26:
        return None, None, None
    
    ema12 = calculate_ema(prices, 12)
    ema26 = calculate_ema(prices, 26)
    
    if ema12 is None or ema26 is None:
        return None, None, None
    
    macd_line = ema12 - ema26
    
    # Calculate signal line (9-day EMA of MACD)
    macd_values = []
    for i in range(26, len(prices)):
        temp_ema12 = calculate_ema(prices[:i+1], 12)
        temp_ema26 = calculate_ema(prices[:i+1], 26)
        if temp_ema12 and temp_ema26:
            macd_values.append(temp_ema12 - temp_ema26)
    
    if len(macd_values) >= 9:
        signal_line = calculate_ema(macd_values, 9)
        histogram = macd_line - signal_line if signal_line else 0
    else:
        signal_line = None
        histogram = None
    
    return macd_line, signal_line, histogram


def calculate_bollinger_bands(prices, period=20, std_dev=2):
    """Calculate Bollinger Bands"""
    if len(prices) < period:
        return None, None, None
    
    sma = calculate_sma(prices, period)
    if sma is None:
        return None, None, None
    
    recent_prices = prices[-period:]
    std = stdev(recent_prices)
    
    upper_band = sma + (std * std_dev)
    lower_band = sma - (std * std_dev)
    
    return upper_band, sma, lower_band


def calculate_atr(high, low, close, period=14):
    """Calculate Average True Range"""
    if len(high) < period + 1 or len(low) < period + 1 or len(close) < period + 1:
        return None
    
    true_ranges = []
    for i in range(1, len(close)):
        tr = max(
            high[i] - low[i],
            abs(high[i] - close[i-1]),
            abs(low[i] - close[i-1])
        )
        true_ranges.append(tr)
    
    if len(true_ranges) < period:
        return None
    
    atr = sum(true_ranges[-period:]) / period
    return atr


def calculate_support_resistance(prices, high, low):
    """Calculate Support and Resistance levels using Pivot Points"""
    if not prices or not high or not low:
        return {'support': None, 'pivot': None, 'resistance': None}
    
    # Use last available data for pivot calculation
    last_high = high[-1]
    last_low = low[-1]
    last_close = prices[-1]
    
    pivot = (last_high + last_low + last_close) / 3
    resistance = (2 * pivot) - last_low
    support = (2 * pivot) - last_high
    
    return {'support': support, 'pivot': pivot, 'resistance': resistance}


def calculate_volume_analysis(volumes, period=10):
    """Calculate volume analysis"""
    if len(volumes) < period:
        return None, None
    
    avg_volume = sum(volumes[-period:]) / period
    current_volume = volumes[-1]
    volume_ratio = current_volume / avg_volume if avg_volume > 0 else 1
    
    return avg_volume, volume_ratio



def calculate_all_indicators(historical_data, current_data):
    """Calculate all technical indicators"""
    try:
        # Check if we have proper historical data
        has_historical = False
        
        # Check for Alpha Vantage format
        if isinstance(historical_data, dict) and 'Time Series (Daily)' in historical_data:
            has_historical = True
            time_series = historical_data['Time Series (Daily)']
        # Check if historical_data is the direct time series dict
        elif isinstance(historical_data, dict) and len(historical_data) > 0:
            # Check if keys look like dates
            first_key = list(historical_data.keys())[0]
            if '-' in str(first_key) and len(str(first_key)) == 10:  # Date format YYYY-MM-DD
                has_historical = True
                time_series = historical_data
        
        # For Indian stocks with limited data
        if not has_historical:
            prices = [current_data['previous_close'], current_data['open'], 
                     current_data['low'], current_data['high'], current_data['price']]
            highs = [current_data['high']] * 5
            lows = [current_data['low']] * 5
            volumes = [current_data.get('volume', 0)] * 5
            
            # Limited indicators
            indicators = {
                'sma': {'20': None, '50': None, '200': None},
                'ema': {'12': None, '26': None},
                'rsi': None,
                'macd': {'line': None, 'signal': None, 'histogram': None},
                'bollinger': {'upper': None, 'middle': None, 'lower': None},
                'atr': None,
                'support_resistance': calculate_support_resistance(prices, highs, lows),
                'volume': {'current': volumes[-1], 'average': volumes[-1], 'ratio': 1.0}
            }
            return indicators
        
        # For US stocks with full historical data
        prices = []
        highs = []
        lows = []
        volumes = []
        
        # Sort dates and extract data
        sorted_dates = sorted(time_series.keys(), reverse=True)[:200]
        sorted_dates.reverse()  # Oldest to newest
        
        for date in sorted_dates:
            day_data = time_series[date]
            prices.append(float(day_data['4. close']))
            highs.append(float(day_data['2. high']))
            lows.append(float(day_data['3. low']))
            volumes.append(int(day_data['5. volume']))
        
        # Calculate all indicators
        indicators = {
            'sma': {
                '20': calculate_sma(prices, 20),
                '50': calculate_sma(prices, 50),
                '200': calculate_sma(prices, 200)
            },
            'ema': {
                '12': calculate_ema(prices, 12),
                '26': calculate_ema(prices, 26)
            },
            'rsi': calculate_rsi(prices, 14),
            'macd': {},
            'bollinger': {},
            'atr': calculate_atr(highs, lows, prices, 14),
            'support_resistance': {},
            'volume': {}
        }
        
        # MACD
        macd_line, signal_line, histogram = calculate_macd(prices)
        indicators['macd'] = {
            'line': macd_line,
            'signal': signal_line,
            'histogram': histogram
        }
        
        # Bollinger Bands
        bb_upper, bb_middle, bb_lower = calculate_bollinger_bands(prices, 20)
        indicators['bollinger'] = {
            'upper': bb_upper,
            'middle': bb_middle,
            'lower': bb_lower
        }
        
        # Support & Resistance
        indicators['support_resistance'] = calculate_support_resistance(prices, highs, lows)
        
        # Volume Analysis
        avg_vol, vol_ratio = calculate_volume_analysis(volumes, 10)
        indicators['volume'] = {
            'current': volumes[-1] if volumes else 0,
            'average': avg_vol,
            'ratio': vol_ratio
        }
        
        return indicators
        
    except Exception as e:
        print(f"Error calculating indicators: {e}")
        import traceback
        traceback.print_exc()
        return None


def generate_signals(stock_data, indicators):
    """Generate buy/sell signals based on technical indicators"""
    signals = {
        'bullish': [],
        'bearish': [],
        'neutral': []
    }
    
    current_price = stock_data['price']
    
    # Moving Average Signals
    sma20 = indicators['sma'].get('20')
    sma50 = indicators['sma'].get('50')
    sma200 = indicators['sma'].get('200')
    
    if sma20 and sma50:
        if current_price > sma20 and sma20 > sma50:
            signals['bullish'].append("Price above SMA-20 & SMA-50 (Bullish trend)")
        elif current_price < sma20 and sma20 < sma50:
            signals['bearish'].append("Price below SMA-20 & SMA-50 (Bearish trend)")
    
    if sma50 and sma200:
        if sma50 > sma200:
            signals['bullish'].append("Golden Cross detected (SMA-50 > SMA-200)")
        elif sma50 < sma200:
            signals['bearish'].append("Death Cross detected (SMA-50 < SMA-200)")
    
    # RSI Signals
    rsi = indicators.get('rsi')
    if rsi:
        if rsi > 70:
            signals['bearish'].append(f"RSI Overbought ({rsi:.1f} > 70)")
        elif rsi < 30:
            signals['bullish'].append(f"RSI Oversold ({rsi:.1f} < 30) - Potential bounce")
        else:
            signals['neutral'].append(f"RSI Neutral ({rsi:.1f})")
    
    # MACD Signals
    macd = indicators['macd']
    if macd.get('line') is not None and macd.get('signal') is not None:
        if macd['line'] > macd['signal']:
            signals['bullish'].append("MACD bullish crossover")
        elif macd['line'] < macd['signal']:
            signals['bearish'].append("MACD bearish crossover")
    
    # Bollinger Bands Signals
    bb = indicators['bollinger']
    if bb.get('upper') and bb.get('lower'):
        if current_price >= bb['upper'] * 0.98:
            signals['bearish'].append("Price near upper Bollinger Band (Overbought)")
        elif current_price <= bb['lower'] * 1.02:
            signals['bullish'].append("Price near lower Bollinger Band (Oversold)")
    
    # Volume Signals
    vol = indicators['volume']
    if vol.get('ratio'):
        if vol['ratio'] > 1.5:
            if stock_data['change'] > 0:
                signals['bullish'].append(f"High volume on price increase ({vol['ratio']:.1f}x avg)")
            else:
                signals['bearish'].append(f"High volume on price decrease ({vol['ratio']:.1f}x avg)")
    
    # Support/Resistance Signals
    sr = indicators['support_resistance']
    if sr.get('pivot'):
        if current_price > sr['pivot']:
            signals['bullish'].append("Price above pivot point")
        else:
            signals['bearish'].append("Price below pivot point")
    
    # Calculate score
    bullish_count = len(signals['bullish'])
    bearish_count = len(signals['bearish'])
    score = bullish_count - bearish_count
    
    # Determine recommendation
    if score >= 4:
        recommendation = "STRONG BUY"
        confidence = "High"
        emoji = "🟢"
    elif score >= 2:
        recommendation = "BUY"
        confidence = "Medium"
        emoji = "🟢"
    elif score >= -1:
        recommendation = "HOLD"
        confidence = "Medium"
        emoji = "🟡"
    elif score >= -3:
        recommendation = "SELL"
        confidence = "Medium"
        emoji = "🔴"
    else:
        recommendation = "STRONG SELL"
        confidence = "High"
        emoji = "🔴"
    
    signals['score'] = score
    signals['recommendation'] = recommendation
    signals['confidence'] = confidence
    signals['emoji'] = emoji
    signals['percentage'] = min(100, max(0, 50 + (score * 10)))
    
    return signals


In [7]:
def format_output_report(stock_data, indicators, signals, ai_analysis, market):
    """Format the comprehensive output report in markdown"""
    currency = "₹" if market == "Indian" else "$"
    
    # Determine price movement
    if stock_data['change'] > 0:
        change_emoji = "📈"
        change_indicator = "▲"
        change_color = "🟢"
    elif stock_data['change'] < 0:
        change_emoji = "📉"
        change_indicator = "▼"
        change_color = "🔴"
    else:
        change_emoji = "➡️"
        change_indicator = "="
        change_color = "🟡"
    
    # Create visual signal bars
    bull_bars = "█" * len(signals['bullish']) + "░" * (9 - len(signals['bullish']))
    bear_bars = "█" * len(signals['bearish']) + "░" * (9 - len(signals['bearish']))
    
    report = f"""
# 📊 Stock Analysis Report

## **{stock_data['symbol']}** | {market} Market
*Generated: {datetime.datetime.now().strftime('%d %B %Y, %I:%M:%S %p')}*

---

## 🎯 RECOMMENDATION

<div align="center">

### {signals['emoji']} **{signals['recommendation']}** {signals['emoji']}

**Confidence:** {signals['confidence']} | **Score:** {signals['score']:+d}/9 | **Strength:** {signals['percentage']}%

</div>

---

## 💰 Price Overview

| Metric | Value | Change |
|--------|-------|--------|
| **Current Price** | **{currency}{stock_data['price']:.2f}** | {change_color} {change_indicator} {abs(stock_data['change']):.2f} ({stock_data['change_percent']}) |
| Previous Close | {currency}{stock_data['previous_close']:.2f} | - |
| Opening Price | {currency}{stock_data['open']:.2f} | - |
| **Day's Range** | {currency}{stock_data['low']:.2f} → {currency}{stock_data['high']:.2f} | {change_emoji} |
| **52-Week Range** | {currency}{stock_data.get('year_low', 0):.2f} → {currency}{stock_data.get('year_high', 0):.2f} | - |"""
    
    # Add volume with status
    if indicators and indicators['volume'].get('ratio'):
        vol_ratio = indicators['volume']['ratio']
        if vol_ratio > 1.5:
            vol_status = "🔥 **HIGH**"
        elif vol_ratio > 0.7:
            vol_status = "✅ Normal"
        else:
            vol_status = "⚠️ Low"
        report += f"\n| **Volume** | {stock_data['volume']:,} | {vol_status} ({vol_ratio:.2f}x avg) |"
    else:
        report += f"\n| **Volume** | {stock_data['volume']:,} | - |"
    
    report += "\n\n---\n"
    
    # Technical Indicators Section
    if indicators:
        # Check if we have actual indicator data or just placeholders
        has_real_indicators = (
            indicators['sma'].get('20') is not None or 
            indicators['rsi'] is not None or 
            indicators['macd'].get('line') is not None
        )
        
        if has_real_indicators:
            report += "\n## 📊 Technical Indicators\n\n"
            
            # Moving Averages
            report += "### 📈 Moving Averages\n\n"
            
            sma20 = indicators['sma'].get('20')
            sma50 = indicators['sma'].get('50')
            sma200 = indicators['sma'].get('200')
            
            report += "| Period | Value | Status | Difference |\n"
            report += "|--------|-------|--------|------------|\n"
            
            if sma20:
                pos20 = "🟢 Above" if stock_data['price'] > sma20 else "🔴 Below"
                diff20 = abs((stock_data['price'] - sma20) / sma20 * 100)
                report += f"| **SMA-20** | {currency}{sma20:.2f} | {pos20} | {diff20:.2f}% |\n"
            else:
                report += f"| **SMA-20** | *Insufficient data* | - | - |\n"
                
            if sma50:
                pos50 = "🟢 Above" if stock_data['price'] > sma50 else "🔴 Below"
                diff50 = abs((stock_data['price'] - sma50) / sma50 * 100)
                report += f"| **SMA-50** | {currency}{sma50:.2f} | {pos50} | {diff50:.2f}% |\n"
            else:
                report += f"| **SMA-50** | *Insufficient data* | - | - |\n"
                
            if sma200:
                pos200 = "🟢 Above" if stock_data['price'] > sma200 else "🔴 Below"
                diff200 = abs((stock_data['price'] - sma200) / sma200 * 100)
                report += f"| **SMA-200** | {currency}{sma200:.2f} | {pos200} | {diff200:.2f}% |\n"
            else:
                report += f"| **SMA-200** | *Insufficient data* | - | - |\n"
            
            # MA Trend
            if sma50 and sma200:
                if sma50 > sma200:
                    report += "\n> 🟢 **GOLDEN CROSS** - Strong bullish momentum detected!\n\n"
                else:
                    report += "\n> 🔴 **DEATH CROSS** - Bearish momentum detected!\n\n"
            
            # Momentum Oscillators
            report += "### ⚡ Momentum Oscillators\n\n"
            
            # RSI
            rsi = indicators.get('rsi')
            if rsi:
                rsi_filled = int(rsi / 5)
                rsi_bar = "█" * rsi_filled + "░" * (20 - rsi_filled)
                
                if rsi > 70:
                    rsi_status = "🔴 **OVERBOUGHT**"
                    rsi_zone = "⚠️ SELL ZONE"
                elif rsi < 30:
                    rsi_status = "🟢 **OVERSOLD**"
                    rsi_zone = "✅ BUY ZONE"
                else:
                    rsi_status = "🟡 Neutral"
                    rsi_zone = "⏸️ HOLD ZONE"
                
                report += f"**RSI (14-day):** {rsi:.2f} | {rsi_status} | {rsi_zone}\n\n"
                report += f"```\n0 [{rsi_bar}] 100\n```\n\n"
            else:
                report += "**RSI (14-day):** *Insufficient data*\n\n"
            
            # MACD
            macd = indicators['macd']
            if macd.get('line') is not None and macd.get('signal') is not None:
                macd_status = "🟢 **BULLISH**" if macd['line'] > macd['signal'] else "🔴 **BEARISH**"
                
                report += "**MACD Analysis:**\n\n"
                report += f"- **MACD Line:** {macd['line']:.3f}\n"
                report += f"- **Signal Line:** {macd['signal']:.3f}\n"
                
                if macd.get('histogram'):
                    hist_trend = "📈 Increasing" if macd['histogram'] > 0 else "📉 Decreasing"
                    report += f"- **Histogram:** {macd['histogram']:.3f} {hist_trend}\n"
                
                report += f"- **Status:** {macd_status}\n\n"
            else:
                report += "**MACD:** *Insufficient data*\n\n"
            
            # Volatility & Bands
            report += "### 📊 Volatility & Bands\n\n"
            
            bb = indicators['bollinger']
            if bb.get('upper') and bb.get('middle') and bb.get('lower'):
                report += "**Bollinger Bands (20-day, 2σ):**\n\n"
                report += f"- **Upper Band:** {currency}{bb['upper']:.2f}\n"
                report += f"- **Middle Band (SMA-20):** {currency}{bb['middle']:.2f}\n"
                report += f"- **Lower Band:** {currency}{bb['lower']:.2f}\n\n"
                
                # Position
                bb_range = bb['upper'] - bb['lower']
                if bb_range > 0:
                    price_pos = (stock_data['price'] - bb['lower']) / bb_range
                    pos_visual = int(price_pos * 20)
                    pos_bar = "░" * max(0, pos_visual - 1) + "●" + "░" * max(0, 20 - pos_visual)
                    
                    if stock_data['price'] >= bb['upper'] * 0.98:
                        bb_signal = "🔴 **Near Upper Band** - Overbought territory"
                    elif stock_data['price'] <= bb['lower'] * 1.02:
                        bb_signal = "🟢 **Near Lower Band** - Oversold territory"
                    else:
                        bb_signal = "🟡 Middle Zone - Neutral position"
                    
                    report += f"**Price Position:** {bb_signal}\n\n"
                    report += f"```\nLower [{pos_bar}] Upper\n```\n\n"
            else:
                report += "**Bollinger Bands:** *Insufficient data*\n\n"
            
            # ATR
            atr = indicators.get('atr')
            if atr:
                atr_percent = (atr / stock_data['price']) * 100
                if atr_percent > 5:
                    vol_level = "🔴 **HIGH**"
                    vol_desc = "Extreme volatility - Wide stop losses recommended"
                elif atr_percent > 2:
                    vol_level = "🟡 **MEDIUM**"
                    vol_desc = "Moderate volatility - Normal conditions"
                else:
                    vol_level = "🟢 **LOW**"
                    vol_desc = "Low volatility - Tight stop losses possible"
                
                report += f"**ATR (14-day):** {currency}{atr:.2f} ({atr_percent:.2f}%)\n"
                report += f"- **Volatility Level:** {vol_level}\n"
                report += f"- **Assessment:** {vol_desc}\n\n"
            
            # Support & Resistance
            report += "### 🎯 Support & Resistance Levels\n\n"
            
            sr = indicators['support_resistance']
            if sr.get('resistance') and sr.get('support'):
                report += "| Level | Price | Distance from Current |\n"
                report += "|-------|-------|----------------------|\n"
                
                to_resistance = ((sr['resistance'] - stock_data['price']) / stock_data['price']) * 100
                to_pivot = ((sr['pivot'] - stock_data['price']) / stock_data['price']) * 100
                to_support = ((sr['support'] - stock_data['price']) / stock_data['price']) * 100
                
                report += f"| 🔴 **Resistance** | {currency}{sr['resistance']:.2f} | {to_resistance:+.2f}% |\n"
                report += f"| ⚖️ **Pivot Point** | {currency}{sr['pivot']:.2f} | {to_pivot:+.2f}% |\n"
                report += f"| 🟢 **Support** | {currency}{sr['support']:.2f} | {to_support:+.2f}% |\n\n"
                
                # Nearest level
                distances = {
                    'Resistance': abs(to_resistance),
                    'Support': abs(to_support),
                    'Pivot': abs(to_pivot)
                }
                nearest_level = min(distances, key=distances.get)
                
                report += f"> **Nearest Level:** {nearest_level} ({distances[nearest_level]:.2f}% away)\n\n"
            
            report += "---\n\n"
        
        else:
            # For Indian stocks - show available data only
            report += "\n## 📊 Available Market Data\n\n"
            report += "> ⚠️ **Note:** Technical indicators (SMA, RSI, MACD, Bollinger Bands, ATR) require historical daily price data which is not available through NSE API.\n\n"
            report += "> Only current trading session data is displayed below. For full technical analysis, historical data from alternative sources would be needed.\n\n"
            
            report += "### 💰 Current Session Data\n\n"
            report += "| Metric | Value |\n"
            report += "|--------|-------|\n"
            report += f"| Current Price | {currency}{stock_data['price']:.2f} |\n"
            report += f"| Previous Close | {currency}{stock_data['previous_close']:.2f} |\n"
            report += f"| Day Open | {currency}{stock_data['open']:.2f} |\n"
            report += f"| Day High | {currency}{stock_data['high']:.2f} |\n"
            report += f"| Day Low | {currency}{stock_data['low']:.2f} |\n"
            report += f"| Day Range | {currency}{stock_data['high'] - stock_data['low']:.2f} |\n"
            
            if stock_data.get('vwap'):
                report += f"| VWAP | {currency}{stock_data['vwap']:.2f} |\n"
            if stock_data.get('upper_circuit'):
                report += f"| Upper Circuit | {currency}{stock_data['upper_circuit']:.2f} |\n"
            if stock_data.get('lower_circuit'):
                report += f"| Lower Circuit | {currency}{stock_data['lower_circuit']:.2f} |\n"
            
            report += "\n### 📅 52-Week Performance\n\n"
            report += "| Metric | Value | Distance |\n"
            report += "|--------|-------|----------|\n"
            
            if stock_data.get('year_high'):
                pct_from_high = ((stock_data['year_high'] - stock_data['price']) / stock_data['year_high']) * 100
                report += f"| 52-Week High | {currency}{stock_data['year_high']:.2f} | {pct_from_high:.1f}% below |\n"
            
            if stock_data.get('year_low'):
                pct_from_low = ((stock_data['price'] - stock_data['year_low']) / stock_data['year_low']) * 100
                report += f"| 52-Week Low | {currency}{stock_data['year_low']:.2f} | {pct_from_low:.1f}% above |\n"
            
            report += "\n---\n\n"
    
    # Signal Summary
    report += "## 🎯 Signal Analysis\n\n"
    
    report += "### 📊 Signal Distribution\n\n"
    report += f"| Signal Type | Count | Visual |\n"
    report += f"|-------------|-------|--------|\n"
    report += f"| 🟢 **Bullish** | {len(signals['bullish'])}/9 | `{bull_bars}` |\n"
    report += f"| 🔴 **Bearish** | {len(signals['bearish'])}/9 | `{bear_bars}` |\n"
    report += f"| 🟡 **Neutral** | {len(signals['neutral'])}/9 | - |\n\n"
    
    # List signals
    if signals['bullish']:
        report += "### ✅ Bullish Indicators\n\n"
        for i, signal in enumerate(signals['bullish'], 1):
            report += f"{i}. {signal}\n"
        report += "\n"
    
    if signals['bearish']:
        report += "### ❌ Bearish Indicators\n\n"
        for i, signal in enumerate(signals['bearish'], 1):
            report += f"{i}. {signal}\n"
        report += "\n"
    
    # Overall trend
    report += "### 📈 Overall Market Trend\n\n"
    
    if signals['score'] > 3:
        overall = "🟢 **STRONG BULLISH**"
        description = "Powerful upward momentum detected. Multiple indicators confirm buying pressure."
        strength_bar = "█████████████████████████" + "░" * 5
    elif signals['score'] > 0:
        overall = "🟢 **BULLISH**"
        description = "Upward pressure building. Positive indicators outweigh negative ones."
        strength_bar = "████████████████░░░░░░░░░░░░░░"
    elif signals['score'] == 0:
        overall = "🟡 **NEUTRAL**"
        description = "Market indecision. Mixed signals suggest consolidation phase."
        strength_bar = "░░░░░░░░░░░░░░░██████░░░░░░░░░░░░░░░"
    elif signals['score'] > -4:
        overall = "🔴 **BEARISH**"
        description = "Downward pressure building. Negative indicators dominate."
        strength_bar = "░░░░░░░░░░░░░░████████████████"
    else:
        overall = "🔴 **STRONG BEARISH**"
        description = "Powerful downward momentum detected. Multiple indicators confirm selling pressure."
        strength_bar = "░" * 5 + "█████████████████████████"
    
    report += f"**Trend:** {overall}\n\n"
    report += f"**Analysis:** {description}\n\n"
    report += f"```\nBearish ←  [{strength_bar}]  → Bullish\n```\n\n"
    
    report += "---\n\n"
    
    # AI Analysis
    report += "## 🤖 AI-Powered Insights\n\n"
    report += f"{ai_analysis}\n\n"
    report += "---\n\n"
    
    # Disclaimer
    report += """## ⚠️ Risk Disclaimer

> **IMPORTANT:** This analysis is for **EDUCATIONAL PURPOSES ONLY**.

- ❌ **NOT financial advice** - Always conduct your own research
- 📊 Past performance does **NOT** guarantee future results
- 💼 Consult a licensed financial advisor before making investment decisions
- ⚠️ Trading and investing involve **substantial risk of loss**
- 📉 You can lose more than your initial investment

**Trade responsibly and never invest more than you can afford to lose.**

---

*Report generated by Enhanced Stock Analysis System*
"""
    
    return report

In [8]:
# ==================== DATA FETCHING FUNCTIONS ====================

def get_indian_stock_data(symbol):
    """Fetch Indian stock data using NSEPython"""
    try:
        clean_symbol = symbol.replace('.NS', '').replace('.BO', '').upper()
        quote_data = nse_eq(clean_symbol)
        
        if not quote_data:
            return None
        
        price_info = quote_data.get('priceInfo', {})
        intraday_data = price_info.get('intraDayHighLow', {})
        week_data = price_info.get('weekHighLow', {})
        
        total_volume = 0
        try:
            security_data = nse_quote(clean_symbol, "trade_info")
            if security_data and 'securityWiseDP' in security_data:
                total_volume = security_data['securityWiseDP'].get('quantityTraded', 0)
        except:
            total_volume = 0
        
        current_price = float(price_info.get('lastPrice', 0))
        previous_close = float(price_info.get('previousClose', 0))
        change = float(price_info.get('change', 0))
        change_percent = float(price_info.get('pChange', 0))
        
        return {
            'symbol': clean_symbol,
            'price': current_price,
            'change': change,
            'change_percent': f"{change_percent:.2f}%",
            'volume': int(total_volume),
            'previous_close': previous_close,
            'open': float(price_info.get('open', 0)),
            'high': float(intraday_data.get('max', 0)),
            'low': float(intraday_data.get('min', 0)),
            'year_high': float(week_data.get('max', 0)),
            'year_low': float(week_data.get('min', 0)),
            'vwap': float(price_info.get('vwap', 0)),
            'lower_circuit': float(price_info.get('lowerCP', 0)),
            'upper_circuit': float(price_info.get('upperCP', 0))
        }
    except Exception as e:
        print(f"Error fetching Indian stock data: {e}")
        return None


def get_us_stock_data(symbol):
    """Fetch US stock data using Alpha Vantage"""
    try:
        params = {
            'function': 'GLOBAL_QUOTE',
            'symbol': symbol,
            'apikey': ALPHA_VANTAGE_KEY
        }
        response = requests.get(ALPHA_VANTAGE_API, params=params, timeout=10)
        data = response.json()
        
        if 'Global Quote' in data and data['Global Quote']:
            quote = data['Global Quote']
            return {
                'symbol': symbol,
                'price': float(quote.get('05. price', 0)),
                'change': float(quote.get('09. change', 0)),
                'change_percent': quote.get('10. change percent', '0%'),
                'volume': int(quote.get('06. volume', 0)),
                'previous_close': float(quote.get('08. previous close', 0)),
                'open': float(quote.get('02. open', 0)),
                'high': float(quote.get('03. high', 0)),
                'low': float(quote.get('04. low', 0)),
                'year_high': 0,
                'year_low': 0
            }
        return None
    except Exception as e:
        print(f"Error fetching US stock data: {e}")
        return None

def get_stock_history(symbol, market='US'):
    """Fetch historical stock data"""
    try:
        if market == 'Indian':
            return None  # No historical data for Indian stocks
        
        params = {
            'function': 'TIME_SERIES_DAILY',
            'symbol': symbol,
            'apikey': ALPHA_VANTAGE_KEY,
            'outputsize': 'full'
        }
        response = requests.get(ALPHA_VANTAGE_API, params=params, timeout=30)  # Increased timeout
        data = response.json()
        
        # Check for Time Series data
        if 'Time Series (Daily)' in data:
            return data['Time Series (Daily)']  # Return the time series directly
        
        # Check for API errors
        if 'Note' in data:
            print(f"Alpha Vantage Note: {data['Note']}")
        if 'Error Message' in data:
            print(f"Alpha Vantage Error: {data['Error Message']}")
        
        return None
    except Exception as e:
        print(f"Error fetching stock history: {e}")
        return None


In [9]:
# ==================== AI ANALYSIS FUNCTIONS ====================

def create_enhanced_prompt(stock_data, indicators, signals, market):
    """Create enhanced prompt for AI analysis"""
    currency = "₹" if market == "Indian" else "$"
    
    prompt = f"""You are an expert stock analyst. Analyze the following comprehensive data and provide actionable insights.

STOCK: {stock_data['symbol']} ({market} Market)
CURRENCY: {currency}

═══════════════════════════════════════════════════════════
PRICE DATA:
═══════════════════════════════════════════════════════════
Current Price: {currency}{stock_data['price']:.2f}
Change: {stock_data['change']:+.2f} ({stock_data['change_percent']})
Day Range: {currency}{stock_data['low']:.2f} - {currency}{stock_data['high']:.2f}
52W Range: {currency}{stock_data.get('year_low', 0):.2f} - {currency}{stock_data.get('year_high', 0):.2f}
Volume: {stock_data['volume']:,}"""

    if indicators and indicators['volume'].get('average'):
        prompt += f" (Avg: {indicators['volume']['average']:,.0f}, Ratio: {indicators['volume']['ratio']:.2f}x)"
    
    prompt += "\n\n═══════════════════════════════════════════════════════════\n"
    prompt += "TECHNICAL INDICATORS:\n"
    prompt += "═══════════════════════════════════════════════════════════\n"
    
    if indicators:
        # Moving Averages
        prompt += "\nMOVING AVERAGES:\n"
        sma20 = indicators['sma'].get('20')
        sma50 = indicators['sma'].get('50')
        sma200 = indicators['sma'].get('200')
        
        if sma20:
            pos = "above" if stock_data['price'] > sma20 else "below"
            prompt += f"- SMA-20:  {currency}{sma20:.2f} | Price is {pos}\n"
        if sma50:
            pos = "above" if stock_data['price'] > sma50 else "below"
            prompt += f"- SMA-50:  {currency}{sma50:.2f} | Price is {pos}\n"
        if sma200:
            pos = "above" if stock_data['price'] > sma200 else "below"
            prompt += f"- SMA-200: {currency}{sma200:.2f} | Price is {pos}\n"
        
        if sma50 and sma200:
            if sma50 > sma200:
                prompt += "- Trend: Golden Cross (Bullish)\n"
            else:
                prompt += "- Trend: Death Cross (Bearish)\n"
        
        # Momentum
        prompt += "\nMOMENTUM:\n"
        rsi = indicators.get('rsi')
        if rsi:
            if rsi > 70:
                status = "Overbought"
            elif rsi < 30:
                status = "Oversold"
            else:
                status = "Neutral"
            prompt += f"- RSI (14): {rsi:.1f} | Status: {status}\n"
        
        macd = indicators['macd']
        if macd.get('line') is not None:
            prompt += f"- MACD: {macd['line']:.2f} | Signal: {macd.get('signal', 0):.2f}\n"
            if macd.get('histogram'):
                prompt += f"- Histogram: {macd['histogram']:.2f}\n"
            crossover = "Bullish" if macd['line'] > macd.get('signal', 0) else "Bearish"
            prompt += f"- Crossover: {crossover}\n"
        
        # Volatility
        prompt += "\nVOLATILITY & BANDS:\n"
        bb = indicators['bollinger']
        if bb.get('upper'):
            prompt += f"- Bollinger Upper: {currency}{bb['upper']:.2f}\n"
            prompt += f"- Bollinger Middle: {currency}{bb['middle']:.2f}\n"
            prompt += f"- Bollinger Lower: {currency}{bb['lower']:.2f}\n"
            
            if stock_data['price'] >= bb['upper'] * 0.98:
                prompt += "- Price Position: Near Upper Band (Overbought zone)\n"
            elif stock_data['price'] <= bb['lower'] * 1.02:
                prompt += "- Price Position: Near Lower Band (Oversold zone)\n"
            else:
                prompt += "- Price Position: Middle Zone\n"
        
        atr = indicators.get('atr')
        if atr:
            vol_level = "High" if atr > stock_data['price'] * 0.05 else "Medium" if atr > stock_data['price'] * 0.02 else "Low"
            prompt += f"- ATR (14): {currency}{atr:.2f} | Volatility: {vol_level}\n"
        
        # Support & Resistance
        prompt += "\nSUPPORT & RESISTANCE:\n"
        sr = indicators['support_resistance']
        if sr.get('resistance'):
            prompt += f"- Resistance: {currency}{sr['resistance']:.2f}\n"
            prompt += f"- Pivot Point: {currency}{sr['pivot']:.2f}\n"
            prompt += f"- Support: {currency}{sr['support']:.2f}\n"
    
    # Signal Summary
    prompt += "\n═══════════════════════════════════════════════════════════\n"
    prompt += "SIGNAL SUMMARY:\n"
    prompt += "═══════════════════════════════════════════════════════════\n"
    prompt += f"Bullish Signals: {len(signals['bullish'])}/9\n"
    prompt += f"Bearish Signals: {len(signals['bearish'])}/9\n"
    prompt += f"Net Signal Score: {signals['score']:+d}\n"
    prompt += f"Technical Recommendation: {signals['recommendation']}\n"
    
    if signals['bullish']:
        prompt += "\nBullish Factors:\n"
        for sig in signals['bullish']:
            prompt += f"  • {sig}\n"
    
    if signals['bearish']:
        prompt += "\nBearish Factors:\n"
        for sig in signals['bearish']:
            prompt += f"  • {sig}\n"
    
    prompt += """
═══════════════════════════════════════════════════════════
ANALYSIS REQUIREMENTS:
═══════════════════════════════════════════════════════════

Provide your analysis in this EXACT structure (keep it concise):

1. MARKET SENTIMENT (2-3 sentences):
   Brief interpretation of current market mood for this stock.

2. TECHNICAL ASSESSMENT (3-4 sentences):
   Evaluate key indicators and trend strength. Note any important confirmations or divergences.

3. KEY FACTORS (3-5 bullet points):
   • Most important factors influencing your decision
   • Be specific and reference actual values

4. RISK ASSESSMENT:
   Risk Level: [LOW / MEDIUM / HIGH]
   Primary Risks: [1-2 sentence explanation]

5. ENTRY/EXIT STRATEGY:
   • Entry Zone: [price range]
   • Target Price: [price with expected % gain]
   • Stop Loss: [price with acceptable % loss]
   • Timeframe: [Short/Medium/Long term]

6. FINAL VERDICT (1 sentence):
   Clear, actionable summary supporting the {signals['recommendation']} recommendation.

Be direct, specific, and actionable. Focus on the data provided."""
    
    return prompt


In [10]:
def analyze_with_claude(stock_data, indicators, signals, market):
    """Analyze stock using Claude API"""
    try:
        prompt = create_enhanced_prompt(stock_data, indicators, signals, market)
        
        message = anthropic_client.messages.create(
            model=CLAUDE_MODEL,
            max_tokens=1500,
            messages=[{"role": "user", "content": prompt}]
        )
        
        return message.content[0].text
    except Exception as e:
        return f"Claude Analysis Error: {str(e)}"


def analyze_with_openai(stock_data, indicators, signals, market):
    """Analyze stock using OpenAI API"""
    try:
        prompt = create_enhanced_prompt(stock_data, indicators, signals, market)
        
        response = openai_client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1500
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"OpenAI Analysis Error: {str(e)}"


def analyze_with_gemini(stock_data, indicators, signals, market):
    """Analyze stock using Gemini API"""
    try:
        prompt = create_enhanced_prompt(stock_data, indicators, signals, market)
        
        model = genai.GenerativeModel(GEMINI_MODEL)
        response = model.generate_content(prompt)
        
        return response.text
    except Exception as e:
        return f"Gemini Analysis Error: {str(e)}"

In [11]:
# ==================== MAIN ANALYSIS FUNCTION ====================

def analyze_stock(symbol, market, ai_model, progress=gr.Progress()):
    """Main function to analyze stock with enhanced indicators"""
    progress(0, desc="🔄 Starting analysis...")
    
    if not symbol:
        return "❌ Please enter a stock symbol"
    
    try:
        # Fetch stock data based on market
        progress(0.2, desc="📊 Fetching stock data...")
        if market == "Indian":
            stock_data = get_indian_stock_data(symbol)
        else:
            stock_data = get_us_stock_data(symbol)
        
        if not stock_data:
            return f"❌ Unable to fetch data for {symbol}. Please check the symbol and try again."
        
        # Fetch historical data
        progress(0.4, desc="📈 Fetching historical data...")
        history_data = get_stock_history(symbol, market)
        
        # Calculate technical indicators
        progress(0.6, desc="🔢 Calculating technical indicators...")
        indicators = calculate_all_indicators(history_data, stock_data)
        
        # Generate signals
        progress(0.7, desc="⚡ Generating buy/sell signals...")
        signals = generate_signals(stock_data, indicators) if indicators else {
            'bullish': [], 'bearish': [], 'neutral': [],
            'score': 0, 'recommendation': 'HOLD', 'confidence': 'Low',
            'emoji': '🟡', 'percentage': 50
        }
        
        # Get AI analysis
        progress(0.8, desc=f"🤖 Getting AI analysis from {ai_model}...")
        if ai_model == "Claude (Anthropic)":
            ai_analysis = analyze_with_claude(stock_data, indicators, signals, market)
        elif ai_model == "GPT-4 (OpenAI)":
            ai_analysis = analyze_with_openai(stock_data, indicators, signals, market)
        else:  # Gemini
            ai_analysis = analyze_with_gemini(stock_data, indicators, signals, market)
        
        # Format comprehensive report
        progress(0.95, desc="📝 Formatting report...")
        report = format_output_report(stock_data, indicators, signals, ai_analysis, market)
        
        progress(1.0, desc="✅ Analysis complete!")
        return report
        
    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        return f"❌ Error analyzing stock: {str(e)}\n\nDetails:\n{error_details}"

In [12]:
# ==================== GRADIO INTERFACE ====================

with gr.Blocks(theme=gr.themes.Soft(), title="Enhanced Stock Analysis System", css="""
    .output-text {
        font-family: 'Courier New', monospace; 
        font-size: 14px; 
        line-height: 1.6;
        max-height: 800px;
        overflow-y: auto;
    }
    .output-text table {
        border-collapse: collapse;
        width: 100%;
        margin: 10px 0;
    }
    .output-text th, .output-text td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
    }
    .output-text th {
        background-color: #f2f2f2;
    }
""") as app:
    gr.Markdown("""
    # 📈 Enhanced Stock Analysis & Prediction System
    ### AI-Powered Technical Analysis with 15+ Indicators
    
    Get comprehensive stock analysis with:
    ✅ 9+ Technical Indicators (SMA, EMA, RSI, MACD, Bollinger Bands, ATR, S&R)
    ✅ Signal-Based Buy/Sell/Hold Recommendations
    ✅ AI-Powered Insights from Claude, GPT-4, or Gemini
    ✅ Real-time Indian (NSE) & US Market Data
    
    ---
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            symbol_input = gr.Textbox(
                label="📊 Stock Symbol",
                placeholder="e.g., RELIANCE, TCS, AAPL, TSLA",
                info="Enter stock ticker (no .NS/.BO suffix for Indian stocks)"
            )
            
            market_radio = gr.Radio(
                choices=["Indian", "US"],
                value="Indian",
                label="🌍 Market",
                info="Select stock market"
            )
            
            ai_model_radio = gr.Radio(
                choices=["Claude (Anthropic)", "GPT-4 (OpenAI)", "Gemini (Google)"],
                value="Claude (Anthropic)",
                label="🤖 AI Model",
                info="Choose AI for analysis"
            )
            
            analyze_btn = gr.Button("🔍 Analyze Stock", variant="primary", size="lg")
            
            gr.Markdown("""
            ---
            ### 📌 Popular Symbols:
            
            **🇮🇳 Indian (NSE):**
            - RELIANCE, TCS, INFY, HDFCBANK
            - SBIN, ITC, TATAMOTORS, WIPRO
            - BHARTIARTL, ASIANPAINT
            
            **🇺🇸 US Markets:**
            - AAPL, TSLA, GOOGL, MSFT
            - AMZN, NVDA, META, NFLX
            
            **Note:** Indian stocks don't need .NS or .BO suffix
            """)
        
        with gr.Column(scale=2):
            output = gr.Markdown(
                value="Click **'🔍 Analyze Stock'** button to start analysis...",
                label="📊 Comprehensive Analysis Report",
                elem_classes="output-text"
            )
    
    # Examples
    gr.Examples(
        examples=[
            ["RELIANCE", "Indian", "Claude (Anthropic)"],
            ["TCS", "Indian", "GPT-4 (OpenAI)"],
            ["INFY", "Indian", "Gemini (Google)"],
            ["HDFCBANK", "Indian", "Claude (Anthropic)"],
            ["AAPL", "US", "Claude (Anthropic)"],
            ["TSLA", "US", "GPT-4 (OpenAI)"],
            ["NVDA", "US", "Gemini (Google)"],
            ["MSFT", "US", "Claude (Anthropic)"],
        ],
        inputs=[symbol_input, market_radio, ai_model_radio],
    )
    
    analyze_btn.click(
        fn=analyze_stock,
        inputs=[symbol_input, market_radio, ai_model_radio],
        outputs=output
    )
    
    gr.Markdown("""
    ---
    ### ℹ️ About Technical Indicators:
    
    - **SMA/EMA**: Trend direction and strength
    - **RSI**: Overbought (>70) / Oversold (<30) conditions
    - **MACD**: Momentum and trend changes
    - **Bollinger Bands**: Volatility and price extremes
    - **ATR**: Volatility measurement for stop-loss
    - **Support/Resistance**: Key price levels
    - **Volume Analysis**: Confirms price movements
    
    **⚠️ Disclaimer:** This tool is for educational purposes only. Not financial advice. Always do your own research and consult a financial advisor before trading.
    """)

In [13]:
app.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\slath\AppData\Local\Temp\ipykernel_4392\1576280545.py:6: DeprecationWarning: The model 'claude-3-5-sonnet-20240620' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  message = anthropic_client.messages.create(
